This is a beautiful soup parser to get the first user written summary on different movies via their imdb ids. --Ethan

In [11]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
import copy
import time
import random
from tqdm.notebook import tqdm

In [12]:
for i in tqdm(range(4)):
  print('test')
  time.sleep(random.uniform(1,4))

  0%|          | 0/4 [00:00<?, ?it/s]

test
test
test
test


In [13]:
csv_input = pd.read_csv('drive/MyDrive/wiki_movie_plots_deduped_c2.csv')

In [14]:
csv_input = csv_input[csv_input.imdb_id != None] # doesn't work. use the following cell

In [15]:
csv_input = csv_input.dropna() #gets rid of rows with None value 

In [16]:
csv_input.sample(10)

,Unnamed: 0,Release Year,Title,Origin,Director,Cast,Genre,Wiki Page,Plot,Revenue,imdb_id,overview,popularity,Revenue_new
10156,10156,1983,Two of a Kind,American,John Herzfeld,"John Travolta, Olivia Newton-John, Charles Dur...",comedy,https://en.wikipedia.org/wiki/Two_of_a_Kind_(1...,"Four angels — Charlie (Charles Durning), Earl ...",23646952.0,tt0086494,God throws his hands up at the foolishness of ...,3.018747,23646952.0
6322,6322,1955,The Kentuckian,American,Burt Lancaster,"Burt Lancaster, Dianne Foster, Diana Lynn",adventure,https://en.wikipedia.org/wiki/The_Kentuckian,"Frontiersman Elias ""Big Eli"" Wakefield (Burt L...",2600000.0,tt0048248,A frontiersman and his son fight to build a ne...,0.786025,0.0
14116,14116,2002,The New Guy,American,Ed Decter,"DJ Qualls, Eliza Dushku, Zooey Deschanel",comedy,https://en.wikipedia.org/wiki/The_New_Guy,The film begins with prison inmate Luther spea...,31167388.0,tt0241760,Nerdy high school senior Dizzy Harrison has fi...,9.255418,0.0
10048,10048,1982,Zapped!,American,Ken Rosenthal,"Scott Baio, Willie Aames",comedy,https://en.wikipedia.org/wiki/Zapped!,"At Ralph Waldo Emerson High School, bookish st...",16897768.0,tt0084945,Peyton and Barney are fun loving high school s...,3.435989,0.0
13902,13902,2001,Life as a House,American,Irwin Winkler,"Kevin Kline, Kristin Scott Thomas, Hayden Chri...",drama,https://en.wikipedia.org/wiki/Life_as_a_House,"George Monroe, an architectural model fabricat...",23903791.0,tt0264796,"When a man is diagnosed with terminal cancer, ...",4.145598,0.0
20913,20913,2005,Keeping Mum,British,Niall Johnson,"Rowan Atkinson, Kristin Scott Thomas",comedy,https://en.wikipedia.org/wiki/Keeping_Mum,"In the opening scene, as young pregnant Rosie ...",18575768.0,tt0444653,A pastor preoccupied with writing the perfect ...,5.742659,18564702.0
3081,3081,1941,How Green Was My Valley,American,John Ford,"Walter Pidgeon, Maureen O'Hara, Roddy McDowall",drama,https://en.wikipedia.org/wiki/How_Green_Was_My...,The movie begins with a monologue by an older ...,6000000.0,tt0033729,At the turn of the century in a Welsh mining v...,9.058578,6000000.0
14481,14481,2004,Hellboy,American,Guillermo del Toro,"Ron Perlman, Doug Jones, Selma Blair, John Hur...",superhero,https://en.wikipedia.org/wiki/Hellboy_(2004_film),"In 1944, with the help of Russian mystic Grigo...",99318987.0,tt0167190,"In the final days of World War II, the Nazis a...",13.206854,99318987.0
13210,13210,1998,Blues Brothers 2000,American,John Landis,"Dan Aykroyd, John Goodman, Joe Morton, J. Evan...",comedy,https://en.wikipedia.org/wiki/Blues_Brothers_2000,Elwood Blues is released from prison after ser...,14051384.0,tt0118747,"Elwood, the now lone ""Blues Brother"" finally r...",13.162168,14051384.0
21022,21022,2008,The Other Man,British,Richard Eyre,"Liam Neeson, Antonio Banderas",drama,https://en.wikipedia.org/wiki/The_Other_Man_(2...,"In Cambridge, Lisa (Linney) is a shoe designer...",970157.0,tt0974613,The story of a husband who suspects his wife o...,4.674662,1143856.0


In [17]:
url_beginning = 'https://www.imdb.com/title/'
url_end = '/plotsummary?ref_=tt_stry_pl'

url_list = []

for imdb_id in csv_input.imdb_id:
  url = copy.copy(url_beginning + str(imdb_id) + url_end)
  url_list.append(url)

In [18]:
print(url_list)
print(len(url_list))

['https://www.imdb.com/title/tt0004635/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0011841/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0012190/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0012349/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0014532/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0014945/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0015881/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0015841/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0015864/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0017925/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0018037/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0019429/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0019591/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0019729/plotsummary?ref_=tt_stry_pl', 'https://www.imdb.com/title/tt0020247/plotsumma

In [33]:
url_list[604]

'https://www.imdb.com/title/tt0050699/plotsummary?ref_=tt_stry_pl'

In [35]:
for i,url in enumerate(url_list):
  if 'tt0066830' in url:
    print(i)
    print(url)

801
https://www.imdb.com/title/tt0066830/plotsummary?ref_=tt_stry_pl


In [20]:
num_errors = 0
num_skips = 0
movie_summaries = []
exp = re.compile('summary-')

In [44]:
url_list[2601]

'https://www.imdb.com/title/tt0114214/plotsummary?ref_=tt_stry_pl'

In [21]:
for i in tqdm(range(2601,len(url_list))):
  try:
    page = requests.get(url_list[i])
    soup = BeautifulSoup(page.content, "html.parser")
    plot_sums = soup.find_all(id = exp)
    if len(plot_sums)>0: # if the list of summaries is not empty, continue. Otherwise, put a flag
      first_summary = plot_sums[0].find('p').string
    else:
      first_summary = '!!!DELETE ME!!!' # no summary was returned and this flag will be used later to delete such rows from the dataset
      num_skips += 1 
  except:
    first_summary = '!!!DELETE ME!!!'
    num_errors += 1
    print('There was an error at '+str(i)+'.')
  
  time.sleep(random.uniform(0,1))
  
  if i % 200 == 0 or i >= len(url_list)-1:
    if i>0: # so it will periodically save in case of an error and will save on the last iteration
      print('Number of pings: '+str(i))
      print('Number of errors: '+str(num_errors))
      print('Number of skips: '+str(num_skips))
      print('Saving file...')
      filepath = '/content/drive/My Drive/imdb_temporary_output_'+str(i)+'.csv'
      with open(filepath, 'w', encoding = 'utf-8-sig') as f:
        df = pd.DataFrame()
        df['imdb_summaries'] = copy.copy(movie_summaries)
        df.to_csv(f)
      print('File saved.')
  movie_summaries.append(first_summary)  
  time.sleep(random.uniform(1,3)) # so the internet police don't come after us for speeding

  0%|          | 0/2750 [00:00<?, ?it/s]

Number of pings: 2800
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 3000
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 3200
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 3400
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 3600
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 3800
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 4000
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 4200
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 4400
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 4600
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 4800
Number of errors: 0
Number of skips: 0
Saving file...
File saved.
Number of pings: 5000
Number of 

In [22]:
movie_summaries

["A chivalrous British officer takes the blame for his cousin's embezzlement and journeys to the American West to start a new life on a cattle ranch.",
 'A naive country girl is tricked into a sham marriage by a wealthy womanizer, then must rebuild her life despite the taint of having borne a child out of wedlock.',
 'An extended family split up in France and Germany find themselves on opposing sides of the battlefield during World War I.']

In [23]:
csv_input

,Unnamed: 0,Release Year,Title,Origin,Director,Cast,Genre,Wiki Page,Plot,Revenue,imdb_id,overview,popularity,Revenue_new,IMDB_Scraped_Summaries


In [17]:
csv_input['IMDB_Scraped_Summaries'] = movie_summaries

In [18]:
# csv_input = csv_input[csv_input.IMDB_Scraped_Summaries != '!!!DELETE ME!!!']

In [19]:
csv_input.to_csv('output.csv', index=False)